<a href="https://colab.research.google.com/github/Giovanna-FP07/Google-Colab/blob/main/Construindo_um_Agente_RAG_do_Zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU pypdf
%pip install -U langchain
%pip install -U langchain-community
%pip install -U langchain-groq
%pip install langchain-huggingface
%pip install langgraph

In [ ]:
from google.colab import userdata
import os
api_key = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = api_key

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
url = 'https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/m2m_strategy_and_objectives_development.pdf'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(url)
pages = []

for page in loader.lazy_load():
  pages.append(page)

In [ ]:
print(f'Metadados: {pages[0].metadata}\n')

In [ ]:
print(f'Resposta: {pages[0].page_content}')

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface  import HuggingFaceEmbeddings

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [ ]:
vector_store = InMemoryVectorStore.from_documents(pages, embed_model)

In [ ]:
docs = vector_store.similarity_search("Objectives Development Process", k=2)

In [ ]:
for doc in docs:
  print(f'Page {doc.metadata["page"]}: {doc.page_content[:300]} \n')

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
template = """
  You are a helpful assistant thay only gives answers bases on the given context.
  If the answer is not in the context, say "I don’t know".

  Context: {context}
  Question: {question}
  Answer:

"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnable import RunnablePassthrough

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = chain.invoke("What are the Objectives Development Process?")
print(f'Response: {response}')